In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18455128
paper_name = 'lain_westwood_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/ranked list of hypersensitivity data.xlsx', sheet_name='Ranked list')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 408 x 7


In [7]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [9]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Plate Location, Unnamed: 2, Unnamed: 3, Hypersensitivity Index, s.e., Gene name]
Index: []


In [11]:
original_data['data'] = -1

In [12]:
original_data.set_index('ORF', inplace=True)
original_data.index.name = 'orf'

In [13]:
original_data = original_data[['data']].copy()

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(407, 1)

# Load & process tested strains

In [16]:
tested = pd.read_excel('raw_data/strain list.xlsx', sheet_name='Strains')

In [17]:
tested['orf'] = tested['ORF'].astype(str)

In [18]:
tested['orf'] = clean_orf(tested['orf'])

In [19]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [21]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [batch, row, column, ORF, gene, orf]
Index: []


In [22]:
tested_orfs = tested['orf'].unique()

In [23]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YGR146W']

In [24]:
tested_orfs = list(tested_orfs) + missing

In [25]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [26]:
original_data.shape

(6062, 1)

# Prepare the final dataset

In [27]:
data = original_data.copy()

In [28]:
dataset_ids = [16497]
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,16497
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16497
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16497       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [36]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [37]:
from IO.save_data_to_db3 import *

In [38]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18455128...
Inserting the new data...


100%|██████████| 1/1 [00:11<00:00, 11.53s/it]

Updating the data_modified_on field...
